In [1]:
import pandas as pd

# Carregando os dados para visualizar as primeiras linhas e entender a estrutura
file_path = '/content/a1-in.csv'
data = pd.read_csv(file_path)

# Visualizando as primeiras linhas para entender a estrutura dos dados
data.head()


,UNIQUEID,SCHOOL,Class,GRADE,CODER,STUDENTID,Gender,OBSNUM,totalobs-forsession,Activity,ONTASK,TRANSITIONS,NumACTIVITIES,FORMATchanges,NumFORMATS,Obsv/act,Transitions/Durations,Total Time
0,14400,B,T9Q,0,Z,600865,0,1,0,Wholecarpet,Y,3,4,1,2,770.5,0.004043,0
1,14401,B,T9Q,0,Z,596466,0,1,1,Wholecarpet,Y,3,4,1,2,770.5,0.004043,23
2,14402,B,T9Q,0,Z,616590,0,1,2,Wholecarpet,Y,3,4,1,2,770.5,0.004043,25
3,14403,B,T9Q,0,Z,734358,1,1,3,Wholecarpet,Y,3,4,1,2,770.5,0.004043,27
4,14404,B,T9Q,0,Z,826308,1,1,4,Wholecarpet,Y,3,4,1,2,770.5,0.004043,31


Estrutura dos dados:

* UNIQUEID: Identificador único para cada observação.
* SCHOOL: A escola onde a observação foi feita.
* Class: A classe onde a observação foi feita.
* GRADE: O grau escolar dos estudantes.
* CODER: Identificador do avaliador que fez a observação.
* STUDENTID: Identificador único do estudante.
* Gender: Gênero do estudante.
* OBSNUM: Número da observação para o estudante.
* totalobs-forsession: Contagem total de observações para a sessão.
* Activity: A atividade que estava sendo realizada.
* ONTASK: Variável alvo, indicando se o estudante estava engajado (Y=Yes) ou distraído (N=No).
* TRANSITIONS: Número de transições de atividades.
* NumACTIVITIES: Número de atividades diferentes observadas.
* FORMATchanges: Mudanças de formato da atividade.
* NumFORMATS: Número de formatos diferentes de atividades.
* Obsv/act: Observação por atividade.
* Transitions/Durations: Razão entre transições e durações.
* Total Time: Tempo total gasto na atividade.



-----


Para prever comportamentos on-task e off-task, consideraremos variáveis que podem influenciar diretamente o engajamento ou a distração dos estudantes. Variáveis como SCHOOL, Class, CODER, e STUDENTID são específicas ao contexto e não contribuem para a generalização do modelo. Portanto, não é necessário considerar.

As variáveis que fazem sentido incluir são:

* GRADE: Pode influenciar o tipo de atividades realizadas e o nível de engajamento.
* Gender: Diferenças de gênero podem influenciar os padrões de engajamento.
* Activity: O tipo de atividade pode ter um grande impacto no engajamento.
* TRANSITIONS, NumACTIVITIES, FORMATchanges, NumFORMATS, Obsv/act, e Transitions/Durations: Variáveis relacionadas à estrutura e dinâmica da atividade.

------

# Segundo o artigo as variáveis utilizadas são:

As variáveis utilizadas como preditoras foram separadas em duas categorias: característica do aluno (gender, grade -- gênero e série) e design instrucional (Activity, Transitions/Durations -- formato da instrução e Transições/Duração).



In [2]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
import numpy as np

# Definindo as variáveis preditoras e a variável alvo
X = data[['GRADE', 'Gender', 'Activity', 'TRANSITIONS', 'NumACTIVITIES', 'FORMATchanges', 'NumFORMATS', 'Obsv/act', 'Transitions/Durations', 'Total Time']]
y = data['ONTASK'].map({'Y': 1, 'N': 0})  # Convertendo a variável alvo para numérica

# Pré-processamento: Codificação de variáveis categóricas e normalização de variáveis numéricas
categorical_features = ['Activity']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

numerical_features = ['GRADE', 'Gender', 'TRANSITIONS', 'NumACTIVITIES', 'FORMATchanges', 'NumFORMATS', 'Obsv/act', 'Transitions/Durations', 'Total Time']
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)])

# Definindo os modelos a serem testados
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'SVM': SVC(),
    'MLP (Neural Network)': MLPClassifier(max_iter=1000)
}

# Realizando a validação cruzada para cada modelo
results = {}
for name, model in models.items():
    # Pipeline completo: Pré-processamento + Modelo
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('model', model)])
    # Validacao cruzada
    cv_scores = cross_val_score(pipeline, X, y, cv=5, scoring='accuracy')
    results[name] = cv_scores

# Exibindo os resultados da validação cruzada
results_df = pd.DataFrame(results).T
results_df['mean_accuracy'] = results_df.mean(axis=1)
results_df['std_accuracy'] = results_df.std(axis=1)
results_df.sort_values(by='mean_accuracy', ascending=False, inplace=True)
results_df

,0,1,2,3,4,mean_accuracy,std_accuracy
Logistic Regression,0.672255,0.668770,0.672016,0.672016,0.672376,0.671487,0.001365
SVM,0.671714,0.523621,0.586008,0.668590,0.603678,0.610722,0.055346
Gradient Boosting,0.664503,0.480527,0.518392,0.518933,0.579156,0.552302,0.064382
MLP (Neural Network),0.638724,0.463397,0.504868,0.495312,0.527227,0.525906,0.060026
Random Forest,0.603029,0.425352,0.472413,0.437793,0.533718,0.494461,0.066033
Decision Tree,0.541374,0.396863,0.410386,0.439776,0.471511,0.451982,0.051555


from matplotlib import pyplot as plt
results_df[0].plot(kind='hist', bins=20, title=0)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
results_df[1].plot(kind='hist', bins=20, title=1)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
results_df[2].plot(kind='hist', bins=20, title=2)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
results_df[3].plot(kind='hist', bins=20, title=3)
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
results_df.plot(kind='scatter', x=3, y=4, s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
results_df[0].plot(kind='line', figsize=(8, 4), title=0)
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
results_df[1].plot(kind='line', figsize=(8, 4), title=1)
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
results_df[2].plot(kind='line', figsize=(8, 4), title=2)
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
results_df[3].plot(kind='line', figsize=(8, 4), title=3)
plt.gca().spines[['top', 'right']].set_visible(False)

In [3]:
# Dividindo os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Criando o pipeline com o pré-processador e o modelo de Regressão Logística
final_model_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                       ('model', LogisticRegression(max_iter=1000))])

# Treinando o modelo com todo o conjunto de treinamento disponível
final_model_pipeline.fit(X_train, y_train)

# Avaliando a performance do modelo no conjunto de teste
test_accuracy = final_model_pipeline.score(X_test, y_test)

test_accuracy

0.6776636019469984

In [4]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

# Previsões no conjunto de teste
y_pred = final_model_pipeline.predict(X_test)
y_pred_proba = final_model_pipeline.predict_proba(X_test)[:, 1]

# Calculando as métricas de classificação
metrics = {
    'Accuracy': accuracy_score(y_test, y_pred),
    'Precision': precision_score(y_test, y_pred),
    'Recall': recall_score(y_test, y_pred),
    'F1 Score': f1_score(y_test, y_pred),
    'AUC': roc_auc_score(y_test, y_pred_proba)
}

# Calculando a matriz de confusão
conf_matrix = confusion_matrix(y_test, y_pred)

metrics, conf_matrix

({'Accuracy': 0.6776636019469984,
  'Precision': 0.6775473399458972,
  'Recall': 1.0,
  'F1 Score': 0.8077832724145345,
  'AUC': 0.5574007848292126},
 array([[   2, 1788],
        [   0, 3757]]))

In [5]:
# Previsões no conjunto de treinamento
y_train_pred = final_model_pipeline.predict(X_train)
y_train_pred_proba = final_model_pipeline.predict_proba(X_train)[:, 1]

# Calculando as métricas de classificação para o conjunto de treinamento
metrics_train = {
    'Accuracy': accuracy_score(y_train, y_train_pred),
    'Precision': precision_score(y_train, y_train_pred),
    'Recall': recall_score(y_train, y_train_pred),
    'F1 Score': f1_score(y_train, y_train_pred),
    'AUC': roc_auc_score(y_train, y_train_pred_proba)
}

metrics_train

{'Accuracy': 0.6707086188243779,
 'Precision': 0.6707240104589307,
 'Recall': 0.9999327911822031,
 'F1 Score': 0.8028925285340386,
 'AUC': 0.5618752207805064}

No conjunto de treinamento, o modelo de Regressão Logística apresentou as seguintes métricas:

* Acurácia: 67.07%
* Precisão: 67.07%
* Recall: 99.99%
* F1 Score: 80.29%
* AUC: 56.19%


Comparando estas métricas com as do conjunto de teste:

* Acurácia no Teste: 67.76%
* Precisão no Teste: 67.75%
* Recall no Teste: 100%
* F1 Score no Teste: 80.78%
* AUC no Teste: 55.74%

----
Você acredita que o seu modelo teve overfitting ou underfitting?

O modelo de Regressão Logística demonstrou não sofrer de overfitting, mantendo desempenho consistente entre treino e teste, indicativo de boa generalização. Apesar de um alto recall, o AUC moderado sugere limitações na discriminação entre classes, apontando para um possível underfitting. Isso implica que, embora o modelo seja estável, há espaço para aprimoramento.